In [1]:
import math
import numpy as np
import scipy.sparse as sp
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

# 1. 학습 설정

In [2]:
config = {
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    
    'submission_path' : "../submission",
    'submission_name' : 'EASE_v1_submission.csv',

    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
    'reg' : 0.001,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

# 2. 데이터 전처리

In [10]:
class MakeMatrixDataSet():
    """
    MatrixDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'train_ratings.csv'))
        
        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('item')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('user')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['item'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['user'].apply(lambda x : self.user_encoder[x])

        self.user_train, self.user_valid = self.generate_sequence_data()

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['time']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def make_matrix(self, user_list, train = True):
        """
        user_item_dict를 바탕으로 행렬 생성
        """
        mat = torch.zeros(size = (user_list.size(0), self.num_item))
        for idx, user in enumerate(user_list):
            if train:
                mat[idx, self.user_train[user.item()]] = 1
            else:
                mat[idx, self.user_train[user.item()] + self.user_valid[user.item()]] = 1
        return mat

    def make_sparse_matrix(self, test = False):
        X = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        for user in self.user_train.keys():
            item_list = self.user_train[user]
            for item in item_list:
                X[user, item] = 1.0
        
        if test:
            for user in self.user_valid.keys():
                item_list = self.user_valid[user]
                for item in item_list:
                    X[user, item] = 1.0

        return X.tocsr()
    

    def make_candidate_item(self, train = True):
        user_pro_df = pd.read_csv(os.path.join(self.config.data_path, 'user_pro.csv'))
        item_pro_df = pd.read_csv(os.path.join(self.config.data_path, 'item_pro.csv'))

        item_pro_df['item'] = item_pro_df['item'].apply(lambda x : self.item_encoder[x])
        item_pro_df['year'] = item_pro_df['year'].astype(int)

        user_pro_df['user'] = user_pro_df['user'].apply(lambda x : self.user_encoder[x])
        user_pro_df['max_year'] = user_pro_df['max_year'].astype(int)
        
        year2item_list = {}
        year_list = user_pro_df['max_year'].unique().tolist()
        for year in year_list:
            item_list = item_pro_df[item_pro_df['year'] <= year + 1]['item'].tolist()
            year2item_list[year + 1] = item_list

        all_item_list = [i for i in range(self.num_item)]
        group_df = user_pro_df.groupby('user')
        candidate = {}
        if train:
            for user, df in group_df:
                max_year = df['max_year'].values[0]
                candidate_item_list = year2item_list[max_year + 1]
                candidate_item_list = set(all_item_list) - set(candidate_item_list)
                candidate_item_list = list(candidate_item_list | set(self.user_train[user]))
                candidate[user] = candidate_item_list
        else:
            for user, df in group_df:
                max_year = df['max_year'].values[0]
                candidate_item_list = year2item_list[max_year + 1]
                candidate_item_list = set(all_item_list) - set(candidate_item_list)
                candidate_item_list = candidate_item_list | set(self.user_train[user])
                candidate_item_list = list(candidate_item_list | set(self.user_valid[user]))
                candidate[user] = candidate_item_list

        return candidate

    def m_s_m(self, candidate):
        X = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        for user in candidate.keys():
            item_list = candidate[user]
            X[user, item_list] = 1.0

        return X

In [11]:
class AEDataSet(Dataset):
    def __init__(self, num_user):
        self.num_user = num_user
        self.users = [i for i in range(num_user)]

    def __len__(self):
        return self.num_user

    def __getitem__(self, idx): 
        user = self.users[idx]
        return torch.LongTensor([user])

# 3. 모델

In [12]:
class EASE():
    def __init__(self, X, reg):
        self.X = self._convert_sp_mat_to_sp_tensor(X)
        self.reg = reg
    
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res
    
    def fit(self):
        '''

        진짜 정말 간단한 식으로 모델을 만듬

        '''
        G = self.X.to_dense().t() @ self.X.to_dense()
        diagIndices = torch.eye(G.shape[0]) == 1
        G[diagIndices] += self.reg

        P = G.inverse()
        B = P / (-1 * P.diag())
        B[diagIndices] = 0

        self.pred = self.X.to_dense() @ B

# 4. 학습 함수

In [13]:
def get_ndcg(pred_list, true_list):
    ndcg = 0
    for rank, pred in enumerate(pred_list):
        if pred in true_list:
            ndcg += 1 / np.log2(rank + 2)
    return ndcg

# hit == recall == precision
def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit

def evaluate(model, X, user_train, user_valid):

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    recon_mat = model.pred.cpu()
    score = recon_mat * torch.from_numpy(1 - X.todense())
    rec_list = score.argsort(dim = 1)

    for user, rec in enumerate(rec_list):
        uv = user_valid[user]
        up = rec[-10:].cpu().numpy().tolist()
        NDCG += get_ndcg(pred_list = up, true_list = uv)
        HIT += get_hit(pred_list = up, true_list = uv)

    NDCG /= len(rec_list)
    HIT /= len(rec_list)

    return NDCG, HIT


def predict(model, X):
    user2rec = {}

    recon_mat = model.pred.cpu()
    score = recon_mat * torch.from_numpy(1 - X.todense())
    rec_list = score.argsort(dim = 1)

    for user, rec in enumerate(rec_list):
        up = rec[-10:].cpu().numpy().tolist()
        user2rec[user] = up
    
    return user2rec

# 5. 학습

In [14]:
make_matrix_data_set = MakeMatrixDataSet(config = config)
user_train, user_valid = make_matrix_data_set.get_train_valid_data()
X = make_matrix_data_set.make_sparse_matrix()

In [15]:
model = EASE(X = X, reg = config.reg)
model.fit()

In [16]:
candidate = make_matrix_data_set.make_candidate_item(train = True)

In [17]:
new_X = make_matrix_data_set.m_s_m(candidate)

In [18]:
ndcg, hit = evaluate(model = model, X = new_X, user_train = user_train, user_valid = user_valid)
print(f'NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

NDCG@10: 0.69471| HIT@10: 0.17233


In [19]:
ndcg, hit = evaluate(model = model, X = X, user_train = user_train, user_valid = user_valid)
print(f'NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

NDCG@10: 0.69426| HIT@10: 0.17225


# 6. 예측

In [21]:
X_test = make_matrix_data_set.make_sparse_matrix(test = True)

In [22]:
model = EASE(X = X_test, reg = config.reg)
model.fit()

In [27]:
user2rec_list = predict(
    model = model, 
    X = X_test,
    )

submision = []
users = [i for i in range(0, make_matrix_data_set.num_user)]
for user in users:
    rec_item_list = user2rec_list[user]
    for item in rec_item_list:
        submision.append(
            {   
                'user' : make_matrix_data_set.user_decoder[user],
                'item' : make_matrix_data_set.item_decoder[item],
            }
        )

submision = pd.DataFrame(submision)

In [28]:
submision.to_csv(os.path.join(config.submission_path, config.submission_name), index=False)

In [29]:
submision

,user,item
0,11,5010
1,11,3081
2,11,3996
3,11,32587
4,11,4886
...,...,...
313595,138493,1704
313596,138493,1270
313597,138493,8970
313598,138493,110
